# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [13]:
# import libraries
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag

import pandas as pd
import numpy as np
from sqlalchemy import create_engine
from sklearn.externals import joblib
import re

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.pipeline import Pipeline
from sklearn.pipeline import FeatureUnion

from sklearn.grid_search import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.neighbors import KNeighborsClassifier

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [2]:
# load data from database
engine = create_engine('sqlite:///MyEngine_Yi.db')
df = pd.read_sql('SELECT * FROM RawData',engine)
X = df.message
Y = df.iloc[:,4:]

### 2. Write a tokenization function to process your text data

In [3]:
def tokenize(text):
    
    # Detect & Replace URL
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    urls = re.findall(url_regex, text)
    for url in urls:
        text = text.replace(url, "urlplaceholder")
        
    # normalization
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # Tokenize text
    words = word_tokenize(text)
    
    # Remove stop words
    words = [w for w in words if w not in stopwords.words("english")]
    
    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    
    clean_words = []
    for word in words:
        clean_word = lemmatizer.lemmatize(word).strip()
        clean_words.append(clean_word)
        
    return clean_words

In [4]:
# check def
for message in X[:5]:
    token = tokenize(message)
    print(message, token)

Weather update - a cold front from Cuba that could pass over Haiti ['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']
Is the Hurricane over or is it not over ['hurricane']
Looking for someone but no name ['looking', 'someone', 'name']
UN reports Leogane 80-90 destroyed. Only Hospital St. Croix functioning. Needs supplies desperately. ['un', 'report', 'leogane', '80', '90', 'destroyed', 'hospital', 'st', 'croix', 'functioning', 'need', 'supply', 'desperately']
says: west side of Haiti, rest of the country today and tonight ['say', 'west', 'side', 'haiti', 'rest', 'country', 'today', 'tonight']


In [5]:
# Remove NaN in Y.
X_clean = X[~Y.isnull().any(axis = 1)]
Y_clean = Y[~Y.isnull().any(axis = 1)]

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [6]:
pipeline = Pipeline([
        ('vect',CountVectorizer(tokenizer = tokenize)),
        ('tfidf',TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [7]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X_clean, Y_clean)

In [ ]:
# Train pipeline
pipeline.fit(X_train,y_train)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [9]:
y_pred = pipeline.predict(X_test)

In [10]:
for i, col in enumerate(y_test.columns):
    print(f'Report for category: {col}:')
    print(classification_report(y_test.iloc[:,i], y_pred[:,i]))

Report for category: related:
             precision    recall  f1-score   support

        0.0       0.31      0.16      0.21      1513
        1.0       0.77      0.89      0.83      4992
        2.0       0.00      0.00      0.00        47

avg / total       0.66      0.72      0.68      6552

Report for category: request:
             precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      5409
        1.0       0.39      0.08      0.13      1143

avg / total       0.76      0.82      0.76      6552

Report for category: offer:
             precision    recall  f1-score   support

        0.0       0.99      1.00      1.00      6510
        1.0       0.00      0.00      0.00        42

avg / total       0.99      0.99      0.99      6552

Report for category: aid_related:
             precision    recall  f1-score   support

        0.0       0.58      0.80      0.67      3748
        1.0       0.45      0.22      0.30      2804

avg / total       0

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [23]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x7f2a3514e620>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   MultiOutputClassifier(estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
               max_depth=None, max_features='auto', max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
               oob_score=False, random_state=None,

In [8]:
parameters = {
    'vect__max_df':[1.0, 2.0],
    #'tfidf__norm':['l1', 'l2']
    'clf__estimator__n_estimators':[10,12],
    'clf__estimator__n_jobs': [1,2]
}

cv = GridSearchCV(pipeline, param_grid = parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [9]:
cv.fit(X_train, y_train)
y_pred2 = cv.predict(X_test)
print(cv.best_params_)

{'clf__estimator__n_estimators': 12, 'clf__estimator__n_jobs': 1, 'vect__max_df': 1.0}


As shown from event rates below, the multi classifer is imbalanced. For most categories (class labels), event rates are below 10%. In such case, F1 score is designed to work well on imbalanced data since it combines precision and recall metrics into a single metric.     
  
Therefore accuracy and F1 score are picked to compare/pick the best classifier. Additionally, for simplicity, let's focus on overall metrics averaged across cateogoies (mean of accuracy, mean of f1 score) for model comparison.

In [10]:
# event rate for each class
(y_test.sum(axis = 0)/y_test.shape[0]).mean()

0.087882410799077465

In [8]:
def Get_Perf(y_test, y_pred):
    """
    Calculate f1 score and accuracy for a classifer.
    
    Input: 
    y_test: true label
    y_pred: model predicted category
    
    Output:
    Dictionary containing following:
    accuracy:         accuracy for each category
    f1:               f1 score for each category
    overall_accuracy: mean of accuracy across categories
    overall_f1:       mean of f1 score across categories
    """
    accuracy = (y_pred == y_test).sum(axis = 0)/y_test.shape[0]
    f1 = dict()
    for i, col in enumerate(y_test.columns):
        precision,recall,fscore,support = score(y_test.iloc[:,i], y_pred[:,i], average='weighted')
        f1[col] = fscore
    f1 = pd.Series(f1)
    
    overall_accuracy = accuracy.mean()
    overall_f1 = f1.mean()
    return {'accuracy':accuracy, 'f1':f1, 'overall_accuracy':overall_accuracy, 'overall_f1':overall_f1}

Show the accuracy, precision, and recall of the tuned model.

In [12]:
# precision, recall
for i, col in enumerate(y_test.columns):
    print(f'Report for category: {col}:')
    print(classification_report(y_test.iloc[:,i], y_pred2[:,i]))

Report for category: related:
             precision    recall  f1-score   support

        0.0       0.35      0.15      0.21      1517
        1.0       0.78      0.92      0.84      4992
        2.0       0.33      0.02      0.04        43

avg / total       0.67      0.73      0.69      6552

Report for category: request:
             precision    recall  f1-score   support

        0.0       0.84      0.97      0.90      5440
        1.0       0.38      0.08      0.13      1112

avg / total       0.76      0.82      0.77      6552

Report for category: offer:
             precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      6527
        1.0       0.00      0.00      0.00        25

avg / total       0.99      1.00      0.99      6552

Report for category: aid_related:
             precision    recall  f1-score   support

        0.0       0.59      0.79      0.68      3849
        1.0       0.44      0.23      0.31      2703

avg / total       0

In [14]:
# Accuracy
perf_grid = Get_Perf(y_test, y_pred2)
print(perf_grid['accuracy'])

related                   0.732753
request                   0.821123
offer                     0.996184
aid_related               0.559371
medical_help              0.915904
medical_products          0.946276
search_and_rescue         0.972070
security                  0.982448
military                  0.970085
child_alone               1.000000
water                     0.931929
food                      0.885073
shelter                   0.906899
clothing                  0.984737
money                     0.976648
missing_people            0.988553
refugees                  0.965507
death                     0.952991
other_aid                 0.853022
infrastructure_related    0.934219
transport                 0.949176
buildings                 0.944902
electricity               0.978632
tools                     0.994353
hospitals                 0.990079
shops                     0.995574
aid_centers               0.986416
other_infrastructure      0.957570
weather_related     

In [15]:
# overall performance
print(f"""Accuracy: {perf_grid['overall_accuracy']:.2%}.\nF1 score: {perf_grid['overall_f1']:.2%}""")

Accuracy: 92.33%.
F1 score: 90.08%


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

#### KNeighborsClassifier

In [19]:
pipeline_knn = Pipeline([
    ('vect', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', MultiOutputClassifier(KNeighborsClassifier()))
])

# train & predict
pipeline_knn.fit(X_train,y_train)
y_pred_knn = pipeline_knn.predict(X_test)

# overall performance
perf_knn = Get_Perf(y_test, y_pred_knn)
print(f"""Accuracy: {perf_knn['overall_accuracy']:.2%}.\nF1 score: {perf_knn['overall_f1']:.2%}""")

Accuracy: 91.97%.
F1 score: 89.55%


#### Add New Feature: Text Length

In [10]:
class Textlen(BaseEstimator, TransformerMixin):
    """Count the length of text in a document."""
    def __init__(self):
        pass
    
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        n_characters = X.str.len()
        return pd.DataFrame(n_characters)


class StartingVerbExtractor(BaseEstimator, TransformerMixin):
    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)  

In [10]:
pipeline_feature = Pipeline([
    ('feature', FeatureUnion([
        
        ('nlp_pipeline', Pipeline([
            ('vect',CountVectorizer(tokenizer = tokenize)),
            ('tfidf',TfidfTransformer())
        ])),
        
        ('txt_len', Textlen())
    ])),
    
    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# train & predict
pipeline_feature.fit(X_train,y_train)
y_pred_feature = pipeline_feature.predict(X_test)

# overall performance
perf_feature = Get_Perf(y_test, y_pred_feature)
print(f"""Accuracy: {perf_feature['overall_accuracy']:.2%}.\nF1 score: {perf_feature['overall_f1']:.2%}""")

Accuracy: 92.36%.
F1 score: 90.03%


Pick the model from grid search.

### 9. Export your model as a pickle file

In [18]:
joblib.dump(cv.best_estimator_,'NLP_model_Yi.pkl')

['NLP_model_Yi.pkl']

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.